In [7]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [44]:
price_data = pd.read_csv("C:/Users/PC/02_Forward School/Capstone Project-Alcon Marshall/00_Datasets/05 Food Price/producer-prices_mys.csv", skiprows=1)

In [48]:
price_data

,#country+code,#date+start,#date+end,Unnamed: 3,Unnamed: 4,#country+name,#indicator+code,Unnamed: 7,#indicator+name,Unnamed: 9,Unnamed: 10,Unnamed: 11,#date+year,Unnamed: 13,Unnamed: 14,#indicator+type,#indicator+value+num,Unnamed: 17
0,MYS,1991-01-01,1991-12-31,131,'458,Malaysia,226,'01379.01,Areca nuts,5539,Producer Price Index (2014-2016 = 100),1991,1991,7021,Annual value,NaN,43.06,I
1,MYS,1992-01-01,1992-12-31,131,'458,Malaysia,226,'01379.01,Areca nuts,5539,Producer Price Index (2014-2016 = 100),1992,1992,7021,Annual value,NaN,43.30,I
2,MYS,1993-01-01,1993-12-31,131,'458,Malaysia,226,'01379.01,Areca nuts,5539,Producer Price Index (2014-2016 = 100),1993,1993,7021,Annual value,NaN,43.91,I
3,MYS,1994-01-01,1994-12-31,131,'458,Malaysia,226,'01379.01,Areca nuts,5539,Producer Price Index (2014-2016 = 100),1994,1994,7021,Annual value,NaN,44.59,I
4,MYS,1995-01-01,1995-12-31,131,'458,Malaysia,226,'01379.01,Areca nuts,5539,Producer Price Index (2014-2016 = 100),1995,1995,7021,Annual value,NaN,45.01,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11594,MYS,2020-01-01,2020-12-31,131,'458,Malaysia,1800,'F1800,"Vegetables&Melons, Total",5539,Producer Price Index (2014-2016 = 100),2020,2020,7021,Annual value,NaN,114.07,I
11595,MYS,2021-01-01,2021-12-31,131,'458,Malaysia,1800,'F1800,"Vegetables&Melons, Total",5539,Producer Price Index (2014-2016 = 100),2021,2021,7021,Annual value,NaN,127.45,I
11596,MYS,2022-01-01,2022-12-31,131,'458,Malaysia,1800,'F1800,"Vegetables&Melons, Total",5539,Producer Price Index (2014-2016 = 100),2022,2022,7021,Annual value,NaN,149.72,I
11597,MYS,2023-01-01,2023-12-31,131,'458,Malaysia,1800,'F1800,"Vegetables&Melons, Total",5539,Producer Price Index (2014-2016 = 100),2023,2023,7021,Annual value,NaN,138.68,I


In [106]:
# Replace inf/-inf with NaN
plant_trends.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values in 'Value'
plant_trends = plant_trends.dropna(subset=["Value"])

In [108]:
# Rename columns manually
price_data.columns = [
    "Iso3", "StartDate", "EndDate", "Area_Code", "Area_Code_M49", "Area",
    "Item_Code", "Item_CPC_Code", "Commodity", "Element_Code", "Element",
    "Year_Code", "Year", "Months_Code", "Months", "Unit", "Value", "Flag"
]

In [110]:
# Filter for Producer Price entries
price_data = price_data[
    price_data["Element"].str.contains("Producer Price", case=False, na=False)
].copy()


In [112]:
# Convert to numeric
price_data["Value"] = pd.to_numeric(price_data["Value"], errors='coerce')
price_data["Year"] = pd.to_numeric(price_data["Year"], errors='coerce')


In [114]:
# Drop rows with missing key values
price_data = price_data.dropna(subset=["Commodity", "Year", "Value"])

In [116]:
# Compute average and round to 0 decimals
avg_prices = price_data.groupby("Commodity")["Value"].mean().reset_index()
avg_prices.columns = ["Commodity", "Avg Producer Price Index"]

In [118]:
# Round values and format with comma separator
avg_prices["Avg Producer Price Index"] = avg_prices["Avg Producer Price Index"].round(0).astype(int)
avg_prices = avg_prices.sort_values(by="Avg Producer Price Index", ascending=False).reset_index(drop=True)

In [132]:
# Format with thousands separator and align right
styled_table = avg_prices.style \
    .format({"Avg Producer Price Index": "{:,.0f}"}) \
    .set_properties(subset=["Avg Producer Price Index"], **{'text-align': 'right'}) \
    .set_caption("Average Producer Price Index by Commodity (2014–2016 = 100)") \
    .hide(axis='index')  # Optional: hide index column for clarity

# Display styled table
display(styled_table)


Commodity,Avg Producer Price Index
"Meat of goat, fresh or chilled (biological)","13,523"
"Meat of sheep, fresh or chilled (biological)","13,202"
"Meat of goat, fresh or chilled","11,899"
"Pepper (Piper spp.), raw","11,861"
Asparagus,"10,452"
"Meat of cattle with the bone, fresh or chilled","9,271"
Unmanufactured tobacco,"8,630"
"Meat of buffalo, fresh or chilled (biological)","8,206"
"Meat of cattle with the bone, fresh or chilled (biological)","8,021"
"Meat of sheep, fresh or chilled","6,419"


In [134]:
styled_highlight = avg_prices.style \
    .format({"Avg Producer Price Index": "{:,.0f}"}) \
    .background_gradient(subset="Avg Producer Price Index", cmap="YlOrRd") \
    .set_caption("Average Producer Price Index by Commodity (Highlight by Price)") \
    .hide(axis='index')

display(styled_highlight)


Commodity,Avg Producer Price Index
"Meat of goat, fresh or chilled (biological)","13,523"
"Meat of sheep, fresh or chilled (biological)","13,202"
"Meat of goat, fresh or chilled","11,899"
"Pepper (Piper spp.), raw","11,861"
Asparagus,"10,452"
"Meat of cattle with the bone, fresh or chilled","9,271"
Unmanufactured tobacco,"8,630"
"Meat of buffalo, fresh or chilled (biological)","8,206"
"Meat of cattle with the bone, fresh or chilled (biological)","8,021"
"Meat of sheep, fresh or chilled","6,419"


In [148]:
# Concatenate top 10 and bottom 10
top_bottom = pd.concat([
    avg_prices.head(10),
    avg_prices.tail(10)
]).reset_index(drop=True)

display(
    top_bottom.style.format({"Avg Producer Price Index": "{:,.0f}"})
    .set_caption("Top and Bottom 10 Commodities by Producer Price Index")
    .hide(axis='index')
)

Commodity,Avg Producer Price Index
Areca nuts,81
Bananas,"1,154"
Bananas cavendish,"1,983"
Cabbages,"1,186"
Carrots and turnips,"1,388"
"Cashew nuts, in shell",80
"Cassava, fresh",767
Cauliflowers and broccoli,"2,471"
"Chillies and peppers, dry (Capsicum spp., Pimenta spp.), raw",79
"Chillies and peppers, green (Capsicum spp. and Pimenta spp.)","4,458"


In [152]:
# Average and style the table
avg_prices = plant_foods.groupby("Commodity")["Value"].mean().reset_index()
avg_prices.columns = ["Commodity", "Avg Producer Price Index"]
avg_prices["Avg Producer Price Index"] = avg_prices["Avg Producer Price Index"].round(0).astype(int)

# Highlighted style with ascending yellow to red color scale
styled_highlight = avg_prices.style \
    .format({"Avg Producer Price Index": "{:,.0f}"}) \
    .background_gradient(subset="Avg Producer Price Index", cmap="YlOrRd") \
    .set_caption("Average Producer Price Index for Plant-Based Whole Foods (2014–2016 = 100)") \
    .hide(axis='index')

display(styled_highlight)


Commodity,Avg Producer Price Index
Areca nuts,81
Bananas,"1,154"
Bananas cavendish,"1,983"
Cabbages,"1,186"
Carrots and turnips,"1,388"
"Cashew nuts, in shell",80
"Cassava, fresh",767
Cauliflowers and broccoli,"2,471"
"Chillies and peppers, dry (Capsicum spp., Pimenta spp.), raw",79
"Chillies and peppers, green (Capsicum spp. and Pimenta spp.)","4,458"


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("producer-prices_mys.csv")

# Filter only Producer Price Index entries
df = df[df['Element'] == 'Producer Price Index (2014-2016 = 100)'].copy()

# Convert 'Value' to numeric and drop invalid entries
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df.dropna(subset=['Value', 'Year'])

# -------------------------------
# Classify Items by Food Category
# -------------------------------

# Define detailed keywords
plant_keywords = [
    "vegetable", "legume", "fruit", "cruciferous", "greens", "beans",
    "nuts", "seeds", "herb", "spices", "whole grain", "maize", "cassava", "ginger",
    "lettuce", "mangoes", "cabbages", "carrots", "broccoli", "cucumbers",
    "cauliflowers", "bananas", "eggplants", "leeks", "coffee", "cocoa", "coconut",
    "pepper", "asparagus", "corn", "turnips", "chillies", "onion", "garlic"
]

animal_keywords = [
    "meat", "egg", "milk", "dairy", "livestock", "buffalo", "cattle", "chicken", "duck", "goat", "pig", "horse"
]

# Categorization function
def classify_item(item):
    item_lower = str(item).lower()
    if any(k in item_lower for k in plant_keywords):
        return "Plant-Based"
    elif any(k in item_lower for k in animal_keywords):
        return "Animal-Based"
    else:
        return "Other"

# Apply classification
df['Category'] = df['Item'].apply(classify_item)

# -------------------------------
# Aggregate by Year and Category
# -------------------------------
grouped = df[df['Category'].isin(["Plant-Based", "Animal-Based"])]
trend_data = grouped.groupby(['Year', 'Category'])['Value'].mean().unstack()

# -------------------------------
# Plotting
# -------------------------------
plt.figure(figsize=(12, 6))
trend_data.plot(marker='o')
plt.title("Time Series of Producer Price Index in Malaysia by Food Category")
plt.ylabel("Producer Price Index (2014–2016 = 100)")
plt.xlabel("Year")
plt.grid(True)
plt.legend(title="Food Category")
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'producer-prices_mys.csv'